In [1]:
import numpy as np
import scipy as sp
from scipy import linalg

- assume double integrator model

In [2]:
# M players
M = 3

# N time steps
N = 5

# n state size
n = 4  # (x, y, v_x, v_y)

# m control input size
m = 2  # (a_x, a_y)

# timestep
dt = 0.01

# collision avoidance radius
r = 0.5

### States

$X^v$ and $U^v$ trajectory for each player $v$ basically like the CFTOCP batch approach without substitution from HW1

$X^v = [x_1, ... x_N]$ and $U^v = [u_0, ... u_{N-1}]$

For the full Nash equilibium problem, just concatenate all the players trajectories together 

$X = [X^1, X^2, ..., X^m]$ and $U = [U^1, U^2, ..., U^m]$



### Dynamics

$D(X,U)$ contains all trajectories and control inputs stacked

In [3]:
# double integrator
A = np.eye(n) + np.diag([dt, dt], k=int(n / 2))
B = np.vstack((np.eye(m) * dt**2 / 2, (np.eye(m) * dt)))

# player dynamics
# A_eq * X_v + B_eq * U_v = E_eq
a_block = linalg.block_diag(*([-A] * (N - 1)))
A_eq = np.eye(N * n) + np.pad(a_block, [(n, 0), (0, n)])
B_eq = linalg.block_diag(*([-B] * N))
E_eq = np.vstack((A, np.zeros(((N - 1) * n, n))))

# system dynamics for all players
# A_sys * X + B_sys * U = E_sys
A_sys = linalg.block_diag(*([A_eq] * M))
B_sys = linalg.block_diag(*([B_eq] * M))
E_sys = np.vstack(([E_eq] * M))

In [4]:
print(np.shape(A_sys), np.shape(B_sys), np.shape(E_sys))
print(np.shape(A_eq), np.shape(B_eq), np.shape(E_eq))
print(np.shape(A), np.shape(B))

(60, 60) (60, 30) (60, 4)
(20, 20) (20, 10) (20, 4)
(4, 4) (4, 2)


### Inequality Constraints

1) walls: all position + radius should not exceed value (assume only horizontal/vertical walls)

$C_{wall}X - (D_{wall} - R) \leq 0$

2) collision avoidance: player position + radius not exceed another player position

$R^2 - ||x_k^v - x_k^{v'} ||_2^2 \leq 0$

3) control input bounds: bounds on control input centered at zero

$-B \leq U \leq B$

In [5]:
# position wall bounds
# example wall at y = 2 (extends forever)
wall_y = 2.0
ind_y = 1  # index corresponding to y position in state

# for single state k
# c_wall_ineq * x_k - d_wall_ineq <= 0
c_wall_ineq = np.zeros(n)
c_wall_ineq[ind_y] = 1
d_wall_ineq = wall_y - r

# for single player
# C_wall_ineq * X_v - D_wall_ineq <= 0
C_wall_ineq = linalg.block_diag(*([c_wall_ineq] * N))
D_wall_ineq = np.array(([d_wall_ineq] * N))

# for all players
# C_wall_sys * X - D_wall_sys <= 0
C_wall_sys = linalg.block_diag(*([C_wall_ineq] * M))
D_wall_sys = np.tile(D_wall_ineq, M)


print(np.shape(D_wall_ineq))
print(np.shape(C_wall_sys), np.shape(D_wall_sys))

(5,)
(15, 60) (15,)


In [6]:
# collision avoidance with other players
# will need to be formulated per timestep k, per players v1, v2
# r - (C_cola_k_v1_v2 * X).T @ (C_cola_k_v1_v2 * X) <= 0

# select position from the states
pos = np.hstack((np.eye(2), np.zeros((2, 2))))

list_cola = []
for k in range(N):  # timestep
    for v1 in range(M):  # player 1
        for v2 in range(v1 + 1, M):  # player 2
            c_block = [np.zeros((2, n))] * N * M
            ind1 = v1 * N + k
            ind2 = v2 * N + k
            c_block[ind1] = pos
            c_block[ind2] = -1 * pos
            C_cola = np.hstack(c_block)
            list_cola.append(C_cola)

# will need to stack up all of these individually after evaluating the quadratic

# there might be better way linear algebra wise to stack them together but prob
# is gonna make the derivative worse to find
np.shape(list_cola)

(15, 2, 60)

In [7]:
# control input bounds
max_x = 5
max_y = 5

ind_x = 0  # index corresponding to x position in control
ind_y = 1
f_ineq = np.zeros((4, m))
f_ineq[0, ind_x] = 1
f_ineq[1, ind_x] = -1
f_ineq[2, ind_y] = 1
f_ineq[3, ind_y] = -1
g_ineq = np.array(([max_x] * 2 + [max_y] * 2))

F_ineq = linalg.block_diag(*([f_ineq] * N))
G_ineq = np.tile(g_ineq, N)

F_sys = linalg.block_diag(*([F_ineq] * M))
G_sys = np.tile(G_ineq, M)

print(np.shape(g_ineq))
print(np.shape(G_ineq))
print(np.shape(F_sys), np.shape(G_sys))

(4,)
(20,)
(60, 30) (60,)


### Objective

In [8]:
# X = np.random.rand(N*M*n)
X = np.array(range(M * N * n))
U = np.array(range(M * N * m)) * 0.1

xf = np.array([1, 1, 2, 2])
Q = np.eye(n) * 2
Qf = np.eye(n) * 5
R = np.eye(m) * 10

u_v = 2

cost = 0
# trajectory states
for k in range(0, N - 2):
    for v in range(M):
        ind = (v * N + k) * n
        xk = X[ind : ind + n]
        cost += 0.5 * (xk - xf).T @ Q @ (xk - xf)

# final state
k = N - 1
ind = (v * N + k) * n
xk = X[ind : ind + n]
cost += 0.5 * (xk - xf).T @ Qf @ (xk - xf)

# player control input
for k in range(0, N - 1):
    ind = (u_v * N + k) * m
    uk = U[ind : ind + m]
    cost += 0.5 * uk.T @ R @ uk
cost

62326.00000000001

In [9]:
# wrt x
xf_sys = np.tile(np.reshape(xf, (n,)), N * M)
Q_sys = linalg.block_diag(*(([Q] * (N - 1) + [Qf]) * M))
J_x = (X - xf_sys).T @ Q_sys

# wrt u
ind = (u_v * N) * m
Uv = U[ind : ind + N * m]
R_sys = linalg.block_diag(*([R] * N))
J_u = Uv.T @ R_sys

np.hstack((J_x, J_u))
print(np.shape(Uv), np.shape(R_sys))
print(np.shape(J_x), np.shape(J_u))
print(np.shape(X), np.shape(U))

(10,) (10, 10)
(60,) (10,)
(60,) (30,)


### Augmented Lagrangian

$ L^v(X,U) = J^v + \mu^{v\top} D + \lambda^\top C + \frac{1}{2}C^\top I_\rho C
$

### Check dimensions

In [10]:
X = np.array(range(M * N * n))
U = np.array(range(M * N * m)) * 0.1
mu = np.random.rand(N * M * n)
y = np.hstack((X, U, mu))

np.split(y, [M * N * n, M * N * (n + m)])

[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
        13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
        26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38.,
        39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51.,
        52., 53., 54., 55., 56., 57., 58., 59.]),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
        1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
        2.6, 2.7, 2.8, 2.9]),
 array([0.50188311, 0.41493488, 0.20448794, 0.00753194, 0.85324509,
        0.22285437, 0.26030263, 0.13637929, 0.03993937, 0.79698965,
        0.60487056, 0.93147713, 0.66047999, 0.68693866, 0.11842266,
        0.71171709, 0.70837931, 0.99090576, 0.25765159, 0.2655233 ,
        0.2477613 , 0.88550294, 0.84910272, 0.64642101, 0.70337452,
        0.04308095, 0.17079275, 0.92300867, 0.5533016 , 0.06051495,
        0.71269485, 0.10990811, 0.32742069, 0.35822511, 0.79976137,
      

In [11]:
print(np.shape(X), np.shape(U))

(60,) (30,)


In [12]:
import dynamics

A_sys, B_sys, E_sys = dynamics.get_system_dynamics(M, N, n, m, dt)
print(np.shape(A_sys), np.shape(B_sys), np.shape(E_sys))

x0 = xf
D = dynamics.D(X, U, A_sys, B_sys, E_sys, x0)
print(np.shape(D))
gD = dynamics.grad_D(X, U, A_sys, B_sys, E_sys)
print(np.shape(gD))

(60, 60) (60, 30) (60, 4)
(60,)
(60, 90)


In [ ]:
import constraints

C_wall_sys, D_wall_sys = constraints.get_system_wall_y(wall_y, r, M, N, n)
print(np.shape(C_wall_sys), np.shape(D_wall_sys))
F_sys, G_sys = constraints.get_system_input_bound(M, N, m, max_x, max_y)
print(np.shape(F_sys), np.shape(G_sys))
list_cola = constraints.get_system_cola(M, N, n)
print(np.shape(list_cola))

c_wall = constraints.C_wall(X, U, C_wall_sys, D_wall_sys)
c_input = constraints.C_input(X, U, F_sys, G_sys)
c_cola = constraints.C_cola(X, U, r, list_cola)

# np.shape(np.concatenate((c_wall,c_input,c_cola)))
C = constraints.C(X, U, C_wall_sys, D_wall_sys, F_sys, G_sys, r, list_cola)
print(np.shape(C))
gC = constraints.grad_C(X, U, C_wall_sys, D_wall_sys, F_sys, G_sys, r, list_cola)
print(np.shape(gC))

(15, 60) (15,)
(60, 30) (60,)
(15, 2, 60)
(90,)
(90, 90)


In [14]:
import objective

u_v = 1
J_v = objective.J_v(X, U, u_v, Q, Qf, R, M, N, n, m, xf)
print(J_v)
gJ_v = objective.grad_J_v(X, U, u_v, Q, Qf, R, M, N, n, m, xf)
print(np.shape(gJ_v))

62178.00000000001
(90,)


In [15]:
import aug_lagrangian

lam = np.ones(len(C))
rho = 1
Irho = aug_lagrangian.Irho(C, lam, rho)
print(np.shape(Irho))

gpenalty = aug_lagrangian.grad_penalty(
    X, U, Irho, C_wall_sys, D_wall_sys, F_sys, G_sys, r, list_cola
)
print(np.shape(gpenalty))

mu = np.ones(M * N * n)
L_v = aug_lagrangian.L_v(u_v, N, n, mu, lam, Irho, J_v, D, C)
print(L_v)

gL_v = aug_lagrangian.grad_L_v(u_v, N, n, m, mu, lam, gJ_v, gD, gC, gpenalty)
print(np.shape(gL_v))

(90, 90)
(90,)
38439.534275000005
(90,)


In [16]:
y = np.concatenate((X, U, mu))
gAL = aug_lagrangian.grad_aug_lagrangian(
    y,
    M,
    N,
    n,
    m,
    lam,
    rho,
    Q,
    Qf,
    R,
    x0,
    xf,
    A_sys,
    B_sys,
    E_sys,
    C_wall_sys,
    D_wall_sys,
    F_sys,
    G_sys,
    r,
    list_cola,
)
print(np.shape(y))
print(np.shape(gAL))

(150,)
(330,)


In [ ]:
from scipy import optimize

al_args = (
    M,
    N,
    n,
    m,
    lam,
    rho,
    Q,
    Qf,
    R,
    x0,
    xf,
    A_sys,
    B_sys,
    E_sys,
    C_wall_sys,
    D_wall_sys,
    F_sys,
    G_sys,
    r,
    list_cola,
)

# currently uses derivative free method
sol = optimize.root(
    aug_lagrangian.grad_aug_lagrangian, y, method="lm", jac=False, args=al_args
)
y = sol.x